# For testing purposes load with Keras to np.array

In [1]:
import densenet

from keras.optimizers import RMSprop
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json
from keras.callbacks import TensorBoard
from keras.applications.xception import Xception

Using TensorFlow backend.


In [2]:
import os, sys
current_dir = os.getcwd()
LESSON_HOME_DIR = current_dir
DATA_HOME_DIR = current_dir+'/data/'
train_data_dir = DATA_HOME_DIR+'train'

In [3]:
import numpy as np
np.random.seed(2016)

import os
import glob
import cv2
import math
import pickle
import datetime
import pandas as pd

from sklearn.cross_validation import train_test_split
from keras.utils import np_utils
from keras.models import model_from_json
from sklearn.metrics import log_loss

/home/paperspace/anaconda2/envs/tensorflow/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
def split_validation_set(train, target, test_size):
    random_state = 51
    X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test

In [5]:
def split_validation_set_with_hold_out(train, target, test_size):
    random_state = 51
    train, X_test, target, y_test = train_test_split(train, target, test_size=test_size, random_state=random_state)
    X_train, X_holdout, y_train, y_holdout = train_test_split(train, target, test_size=test_size, random_state=random_state)
    return X_train, X_test, X_holdout, y_train, y_test, y_holdout

# Another approach to loading the data

In [6]:
from PIL import ImageFilter, ImageStat, Image, ImageDraw
from multiprocessing import Pool, cpu_count
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import glob
import cv2

In [7]:
def im_multi(path):
    try:
        im_stats_im_ = Image.open(path)
        return [path, {'size': im_stats_im_.size}]
    except:
        print(path)
        return [path, {'size': [0,0]}]

def im_stats(im_stats_df):
    im_stats_d = {}
    p = Pool(cpu_count())
    ret = p.map(im_multi, im_stats_df['path'])
    for i in range(len(ret)):
        im_stats_d[ret[i][0]] = ret[i][1]
    im_stats_df['size'] = im_stats_df['path'].map(lambda x: ' '.join(str(s) for s in im_stats_d[x]['size']))
    return im_stats_df

def get_im_cv2(path):
    img = cv2.imread(path)
    resized = cv2.resize(img, (32, 32), cv2.INTER_LINEAR) #use cv2.resize(img, (64, 64), cv2.INTER_LINEAR)
    return [path, resized]

def normalize_image_features(paths):
    imf_d = {}
    p = Pool(cpu_count())
    ret = p.map(get_im_cv2, paths)
    for i in range(len(ret)):
        imf_d[ret[i][0]] = ret[i][1]
    ret = []
    fdata = [imf_d[f] for f in paths]
    fdata = np.array(fdata, dtype=np.uint8)
    fdata = fdata.transpose((0, 3, 1, 2))
    fdata = fdata.astype('float32')
    fdata = fdata / 255
    return fdata

In [8]:
train = glob.glob('data/train/**/*.jpg')# + glob.glob('../input/additional/**/*.jpg')

In [9]:
train[1]

'data/train/Type_3/1127.jpg'

In [10]:
train[1].split('/')

['data', 'train', 'Type_3', '1127.jpg']

In [11]:
train = pd.DataFrame([[p.split('/')[2],p.split('/')[3],p] for p in train], columns = ['type','image','path'])#[::5] #limit for Kaggle Demo

In [12]:
train = im_stats(train)
train = train[train['size'] != '0 0'].reset_index(drop=True) #remove bad images
train_data = normalize_image_features(train['path'])
np.save('train.npy', train_data, allow_pickle=True, fix_imports=True)

In [30]:
le = LabelEncoder()
train_target = le.fit_transform(train['type'].values)
train_target = np_utils.to_categorical(train_target, num_classes=3)

In [32]:
print(train_target[:5])

[[ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]]


In [31]:
print(le.classes_) #in case not 1 to 3 order
np.save('train_target.npy', train_target, allow_pickle=True, fix_imports=True)

['Type_1' 'Type_2' 'Type_3']


Expected input to be images (as Numpy array) following the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (1480, 3, 32, 32) (32 channels).

In [33]:
print(train_data.shape)

(1480, 3, 32, 32)


In [34]:
train_data_channels_last = np.transpose(train_data, (0, 2, 3, 1))
# see: https://stackoverflow.com/a/23944468/6941138

In [35]:
print(train_data_channels_last.shape)

(1480, 32, 32, 3)


## Splitting the data

In [36]:
X_train, X_val, Y_train, Y_val = split_validation_set(train_data_channels_last, train_target, 0.1)

In [37]:
print(X_train.shape)

(1332, 32, 32, 3)


# Now building the model

In [38]:
def printTrainableLayers(model):
    for i, layer in enumerate(model.layers):
       if layer.trainable == True: 
           print(i, layer.name)

In [39]:
def printAllLayers(model):
    for i, layer in enumerate(model.layers):
       print(i, layer.name)

In [40]:
base_model = Xception(weights='imagenet', include_top=False)

In [41]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu', kernel_initializer='glorot_uniform')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu', kernel_initializer='glorot_uniform')(x)
# and a logistic layer -- let's say we have 200 classes
x = Dropout(0.5)(x)
predictions = Dense(3, activation='softmax', kernel_initializer='glorot_uniform')(x)

In [42]:
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [21]:
printAllLayers(model)

(0, 'input_1')
(1, 'block1_conv1')
(2, 'block1_conv1_bn')
(3, 'block1_conv1_act')
(4, 'block1_conv2')
(5, 'block1_conv2_bn')
(6, 'block1_conv2_act')
(7, 'block2_sepconv1')
(8, 'block2_sepconv1_bn')
(9, 'block2_sepconv2_act')
(10, 'block2_sepconv2')
(11, 'block2_sepconv2_bn')
(12, 'conv2d_1')
(13, 'block2_pool')
(14, 'batch_normalization_1')
(15, 'add_1')
(16, 'block3_sepconv1_act')
(17, 'block3_sepconv1')
(18, 'block3_sepconv1_bn')
(19, 'block3_sepconv2_act')
(20, 'block3_sepconv2')
(21, 'block3_sepconv2_bn')
(22, 'conv2d_2')
(23, 'block3_pool')
(24, 'batch_normalization_2')
(25, 'add_2')
(26, 'block4_sepconv1_act')
(27, 'block4_sepconv1')
(28, 'block4_sepconv1_bn')
(29, 'block4_sepconv2_act')
(30, 'block4_sepconv2')
(31, 'block4_sepconv2_bn')
(32, 'conv2d_3')
(33, 'block4_pool')
(34, 'batch_normalization_3')
(35, 'add_3')
(36, 'block5_sepconv1_act')
(37, 'block5_sepconv1')
(38, 'block5_sepconv1_bn')
(39, 'block5_sepconv2_act')
(40, 'block5_sepconv2')
(41, 'block5_sepconv2_bn')
(42, 'b

In [43]:
# set the first 310 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:132]:
    layer.trainable = False

In [44]:
printTrainableLayers(model)

(132, 'global_average_pooling2d_2')
(133, 'dense_4')
(134, 'dropout_3')
(135, 'dense_5')
(136, 'dropout_4')
(137, 'dense_6')


In [45]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
block1_conv1 (Conv2D)            (None, None, None, 32 864         input_2[0][0]                    
____________________________________________________________________________________________________
block1_conv1_bn (BatchNormalizat (None, None, None, 32 128         block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_conv1_act (Activation)    (None, None, None, 32 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [46]:
filepath="Xception-top-improvement_learning_rate_decy_0.01-{epoch:02d}-{val_loss:}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
tbCallBack = TensorBoard(log_dir='./graph_xcep_top', histogram_freq=0, write_graph=True, write_images=True)
callbacks_list.append(tbCallBack)

In [47]:
opt = RMSprop(lr=1e-2, rho=0.9, decay=0.01)
#opt = SGD(lr=1e-2, momentum=0.9, decay=0.01, nesterov=True)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [48]:
batch_size = 16
nb_classes = 3
nb_epoch = 120
nb_train_samples = X_train.shape[0]
nb_val_samples = X_val.shape[0]

In [49]:
datagen = ImageDataGenerator(
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.2,
        fill_mode='nearest')
datagen.fit(train_data_channels_last)

In [50]:
train_generator = datagen.flow(X_train,Y_train)

Sample hat shape: (32,32,32,3)

In [108]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=nb_epoch,
        validation_data=(X_val, Y_val),
        validation_steps=nb_val_samples // batch_size,
        callbacks=callbacks_list)

ValueError: Error when checking input: expected input_3 to have shape (None, 299, 299, 3) but got array with shape (148, 32, 32, 3)

In [109]:
# serialize model to JSON
model_json = model.to_json()
with open("xcep_top_only.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("xcep_top_only.h5")
print("Saved model to disk")

Saved model to disk


In [83]:
#history = model.fit_generator(, nb_epoch=nb_epoch, verbose=20, callbacks=callbacks_list, validation_data=(X_val, Y_val), initial_epoch=0)

/home/paperspace/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., initial_epoch=0, callbacks=[<keras.ca..., epochs=120, validation_data=(array([[[..., verbose=20)`
  """Entry point for launching an IPython kernel.


TypeError: fit_generator() takes at least 3 arguments (7 given)

In [27]:
#history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch, verbose=1, callbacks=callbacks_list, validation_data=(X_test, Y_test), shuffle=True, initial_epoch=0)

ValueError: Error when checking target: expected dense_3 to have shape (None, 3) but got array with shape (1198, 1)